<a href="https://colab.research.google.com/github/ali-sdg/Image_colorization/blob/main/od1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive


drive.mount("/content/drive/")
img=cv2.imread("/content/drive/MyDrive/Colab Notebooks/Data/image processing/od1/data/00056v.jpg")
plt.imshow(img)



In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


H, W = img_gray.shape
third = H // 3


# B kanali
B = img_gray[0:third, :]

# G kanali
G = img_gray[third:2*third, :]

# R kanali
R = img_gray[2*third:3*third, :]

# 4. kanalari gostermk
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(B, cmap="gray")
plt.title("B Channel (Top)")

plt.subplot(1, 3, 2)
plt.imshow(G, cmap="gray")
plt.title("G Channel (Middle)")

plt.subplot(1, 3, 3)
plt.imshow(R, cmap="gray")
plt.title("R Channel (Bottom)")
plt.show()

In [ ]:
# ayarları yapıyorum
max_shfit = 15 # Maksimum kaydırma (x ve y için)
best_ssd_g = 9999999999999999.0 # Başlangıçta en yuksek sayi olsun, en iyi ssd aramak için
best_dy_g, best_dx_g = 0, 0
bordr_crop = 10 # kenarlari kesmek lazım

# Referans kanali B'yi kes
# B_crop zaten B kanalin boyutunda olmali (third x W)
h_crop = third - 2 * bordr_crop
w_crop = W - 2 * bordr_crop
B_crop = B[bordr_crop:third-bordr_crop, bordr_crop:W-bordr_crop] # B kanal referans

# G için arama baslasin
print("G kanali için ahratma basliyor...")

for dy in range(-max_shfit, max_shfit + 1):
    for dx in range(-max_shfit, max_shfit + 1):

        # 1. G'yi elle (manuel) kaydırma (Shift) yapiyorum

        G_shiftd = np.zeros_like(G)

        # Kaynaktaki ve hedefteki koordinatları hesaplama
        src_row_s = max(0, -dy) # Sütun başlangıcı kaynak
        src_row_e = G.shape[0] + min(0, -dy) # Sütun sonu kaynak
        dest_row_s = max(0, dy) # Sütun başlangıcı hedef
        dest_row_e = G.shape[0] + min(0, dy) # Sütun sonu hedef

        src_col_s = max(0, -dx)
        src_col_e = G.shape[1] + min(0, -dx)
        dest_col_s = max(0, dx)
        dest_col_e = G.shape[1] + min(0, dx)

        # Görüntü parçasi G_shiftd'e kopya
        G_shiftd[dest_row_s:dest_row_e, dest_col_s:dest_col_e] = \
            G[src_row_s:src_row_e, src_col_s:src_col_e]


        # 2. G kanalindan kenarlari kes
        G_shiftd_crop = G_shiftd[bordr_crop:third-bordr_crop, bordr_crop:W-bordr_crop]

        # 3. SSD hesapla
        # farklari kare yapip topla
        ssd_g_current = np.sum((B_crop.astype(float) - G_shiftd_crop.astype(float)) ** 2)

        # 4. En iyi sonucu güncelliyorum
        if ssd_g_current < best_ssd_g:
            best_ssd_g = ssd_g_current
            best_dy_g, best_dx_g = dy, dx

print(f"En iyi G kaydirma: dy={best_dy_g}, dx={best_dx_g}") # sonuçlari yazdir

# G kanalina nihayi kaydirmayi uygula
G_algnd = np.roll(G, shift=(best_dy_g, best_dx_g), axis=(0, 1)) # hizli kaydirma

In [ ]:
best_ssd_r = 999999999999.0 # Büyuk bir sayi
best_dy_r, best_dx_r = 0, 0 # en iyi kaydirma sifir olsun şimdilik

for dy in range(-max_shfit, max_shfit + 1):
    for dx in range(-max_shfit, max_shfit + 1):

        # 1. R'yi kaydirma
        R_shiftd = np.zeros_like(R)

        # koordinatları tekrar hesapa
        src_row_s = max(0, -dy)
        src_row_e = R.shape[0] + min(0, -dy)
        dest_row_s = max(0, dy)
        dest_row_e = R.shape[0] + min(0, dy)

        src_col_s = max(0, -dx)
        src_col_e = R.shape[1] + min(0, -dx)
        dest_col_s = max(0, dx)
        dest_col_e = R.shape[1] + min(0, dx)

        # R parçasını kopyala
        R_shiftd[dest_row_s:dest_row_e, dest_col_s:dest_col_e] = \
            R[src_row_s:src_row_e, src_col_s:src_col_e]


        # 2. R kanalindan kenarlari kes
        R_shiftd_crop = R_shiftd[bordr_crop:third-bordr_crop, bordr_crop:W-bordr_crop]

        # 3. SSD hesapla
        # Farklarin karelerinin toplami
        ssd_r_current = np.sum((B_crop.astype(float) - R_shiftd_crop.astype(float)) ** 2)

        # 4. En iyi sonucu bul
        if ssd_r_current < best_ssd_r:
            best_ssd_r = ssd_r_current
            best_dy_r, best_dx_r = dy, dx

print(f"En iyi R kaydirma: dy={best_dy_r}, dx={best_dx_r}")

# R kanalina nihayi kaydirma
R_algnd = np.roll(R, shift=(best_dy_r, best_dx_r), axis=(0, 1))

In [ ]:
# Kanallari birlestiri (R, G, B sirasi)
# Bu bizim tmazlanmiş sonucumuz
tmazlanmis_rgb = cv2.merge([R_algnd, G_algnd, B])

# Sonucu
plt.figure(figsize=(9, 9))
plt.imshow(tmazlanmis_rgb)
plt.title("Sonuc") # Temzlanmis
plt.show()

In [ ]:

# 1. RGB'den HSV'ye çeviri, çünkü sadece parlaklik (V) kanalını değiştirmek istiyor
hsv_goruntu = cv2.cvtColor(tmazlanmis_rgb, cv2.COLOR_RGB2HSV)

# 2. Kanallari ayir (H, S, V)
H_kanali, S_kanali, V_kanali = cv2.split(hsv_goruntu)

# 3. V kanali üzerinde Histagram Esitleme
V_esitlenmis = cv2.equalizeHist(V_kanali)

# 4. Kanallari tekrar birlestirmel
hsv_artirilmis = cv2.merge([H_kanali, S_kanali, V_esitlenmis])

# 5. Sonucu tekrar RGB'ye çevirmek
artirilmis_rgb = cv2.cvtColor(hsv_artirilmis, cv2.COLOR_HSV2RGB)

# Kontrol için Histagramlari çizdirmek
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
hst_asil = cv2.calcHist([V_kanali],[0],None,[50],(0 ,256))
plt.plot(hst_asil, color='red')
plt.title("Asıl V Histagram") # Asil V

plt.subplot(1, 2, 2)
hst_esit = cv2.calcHist([V_esitlenmis],[0],None,[50],(0 ,256))
plt.plot(hst_esit, color='blue')
plt.title("Esitlenmiş V Histagram") # Esitlenmis V
plt.show()

In [ ]:
# Nihayi sonucu
plt.figure(figsize=(9, 9))
plt.imshow(artirilmis_rgb)
plt.title(" Sonuc ")
plt.show()